In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import os
from PIL import Image
import logging
from datetime import datetime

# Setup logging
log_file = f"Ami-br_efficientnet_master_baseline.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet default size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load data from folders
def load_data_from_folder(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Specify the classes
classes = ['Atypical', 'Normal']

# Load dataset
images, labels = load_data_from_folder(
    '/data/Mitosis_Detection/MICCAI25/AMI-Br_dataset/Train/Images', 
    classes
)

# Define the EfficientNet model for binary classification
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Training configuration
num_epochs = 100
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCELoss()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform K-Fold Cross Validation
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(images, labels)):
    logger.info(f"Starting Fold {fold + 1}")

    # Split the raw lists into training and validation parts
    train_images = [images[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_images = [images[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    # Create separate Datasets with different transforms
    train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)
    val_dataset = CustomDataset(val_images, val_labels, transform=val_transform)

    # Calculate class weights for WeightedRandomSampler
    class_counts = [0] * len(classes)
    for lbl in train_labels:
        class_counts[lbl] += 1
    class_weights = [1.0 / count for count in class_counts]

    sample_weights = [class_weights[lbl] for lbl in train_labels]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Initialize model, optimizer, and scheduler
    model = BinaryEfficientNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    best_val_balanced_accuracy = 0.0
    best_model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_targets = []

        # Training loop
        for images_batch, labels_batch in tqdm(train_loader, 
                                               desc=f"Fold {fold + 1} - Epoch {epoch + 1}/{num_epochs} - Training"):
            images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
            labels_batch = labels_batch.float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend((outputs > 0.5).cpu().numpy())
            train_targets.extend(labels_batch.cpu().numpy())

        train_balanced_accuracy = balanced_accuracy_score(train_targets, train_preds)

        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []

        # Validation loop
        with torch.no_grad():
            for images_batch, labels_batch in tqdm(val_loader, 
                                                   desc=f"Fold {fold + 1} - Epoch {epoch + 1}/{num_epochs} - Validation"):
                images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
                labels_batch = labels_batch.float().unsqueeze(1)

                outputs = model(images_batch)
                loss = criterion(outputs, labels_batch)

                val_loss += loss.item()
                val_preds.extend((outputs > 0.5).cpu().numpy())
                val_targets.extend(labels_batch.cpu().numpy())

        val_balanced_accuracy = balanced_accuracy_score(val_targets, val_preds)

        if val_balanced_accuracy > best_val_balanced_accuracy:
            best_val_balanced_accuracy = val_balanced_accuracy
            torch.save(model.state_dict(), best_model_path)

        scheduler.step()

        # Logging metrics
        log_entry = (
            f"Fold {fold + 1}, Epoch {epoch + 1}/{num_epochs} | "
            f"Train Loss: {train_loss / len(train_loader):.4f}, "
            f"Train Balanced Accuracy: {train_balanced_accuracy:.4f} | "
            f"Val Loss: {val_loss / len(val_loader):.4f}, "
            f"Val Balanced Accuracy: {val_balanced_accuracy:.4f}"
        )
        logger.info(log_entry)

    logger.info(f"Fold {fold + 1} - Best Validation Balanced Accuracy: {best_val_balanced_accuracy:.4f}")
    fold_accuracies.append(best_val_balanced_accuracy)

# Calculate and log overall performance
average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
logger.info(f"Average Validation Balanced Accuracy across all folds: {average_accuracy:.4f}")


2025-02-15 19:09:52,976 - INFO - Starting Fold 1


Loaded pretrained weights for efficientnet-b0


Fold 1 - Epoch 1/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.30it/s]
2025-02-15 19:09:58,375 - INFO - Fold 1, Epoch 1/100 | Train Loss: 0.6516, Train Balanced Accuracy: 0.6128 | Val Loss: 0.8555, Val Balanced Accuracy: 0.6284
Fold 1 - Epoch 2/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.24it/s]
2025-02-15 19:10:02,481 - INFO - Fold 1, Epoch 2/100 | Train Loss: 0.4851, Train Balanced Accuracy: 0.7696 | Val Loss: 1.6584, Val Balanced Accuracy: 0.6970
Fold 1 - Epoch 3/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  9.08it/s]
2025-02-15 19:10:06,498 - INFO - Fold 1, Epoch 3/100 | Train Loss: 0.3747, Train Balanced Accuracy: 0.8398 | Val Loss: 1.3641, Val Balanced Accuracy: 0.7573
Fold 1 - Epoch 4/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.90it/s]
2025-02-15 19:10:10,502 - INFO - Fold 1, Epoch 4/100 | Train Loss: 0.3205, Train Balanced Accuracy: 0.8693 | Val Loss: 0.9380, Val Balanced Accuracy: 0.7226
Fold 1 - Epoch 5/100 - Validation: 100%|████████

Loaded pretrained weights for efficientnet-b0


Fold 2 - Epoch 1/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.49it/s]
2025-02-15 19:16:52,595 - INFO - Fold 2, Epoch 1/100 | Train Loss: 0.6317, Train Balanced Accuracy: 0.6275 | Val Loss: 1.6802, Val Balanced Accuracy: 0.5224
Fold 2 - Epoch 2/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.09it/s]
2025-02-15 19:16:56,865 - INFO - Fold 2, Epoch 2/100 | Train Loss: 0.4899, Train Balanced Accuracy: 0.7663 | Val Loss: 0.9730, Val Balanced Accuracy: 0.7208
Fold 2 - Epoch 3/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.60it/s]
2025-02-15 19:17:01,169 - INFO - Fold 2, Epoch 3/100 | Train Loss: 0.3909, Train Balanced Accuracy: 0.8284 | Val Loss: 0.7073, Val Balanced Accuracy: 0.7472
Fold 2 - Epoch 4/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.10it/s]
2025-02-15 19:17:05,400 - INFO - Fold 2, Epoch 4/100 | Train Loss: 0.3131, Train Balanced Accuracy: 0.8739 | Val Loss: 0.5843, Val Balanced Accuracy: 0.7437
Fold 2 - Epoch 5/100 - Validation: 100%|████████

...

Fold 3 - Epoch 1/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.51it/s]
2025-02-15 19:23:44,830 - INFO - Fold 3, Epoch 1/100 | Train Loss: 0.6045, Train Balanced Accuracy: 0.6487 | Val Loss: 2.0061, Val Balanced Accuracy: 0.5986
Fold 3 - Epoch 2/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.04it/s]
2025-02-15 19:23:48,961 - INFO - Fold 3, Epoch 2/100 | Train Loss: 0.4552, Train Balanced Accuracy: 0.7898 | Val Loss: 1.6228, Val Balanced Accuracy: 0.6249
Fold 3 - Epoch 3/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.80it/s]
2025-02-15 19:23:53,083 - INFO - Fold 3, Epoch 3/100 | Train Loss: 0.3492, Train Balanced Accuracy: 0.8436 | Val Loss: 1.4301, Val Balanced Accuracy: 0.6799
Fold 3 - Epoch 4/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.88it/s]
2025-02-15 19:23:57,099 - INFO - Fold 3, Epoch 4/100 | Train Loss: 0.2868, Train Balanced Accuracy: 0.8815 | Val Loss: 0.8733, Val Balanced Accuracy: 0.6734
Fold 3 - Epoch 5/100 - Validation: 100%|████████

Loaded pretrained weights for efficientnet-b0


Fold 4 - Epoch 1/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.84it/s]
2025-02-15 19:30:36,511 - INFO - Fold 4, Epoch 1/100 | Train Loss: 0.6321, Train Balanced Accuracy: 0.6262 | Val Loss: 1.8731, Val Balanced Accuracy: 0.5066
Fold 4 - Epoch 2/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.80it/s]
2025-02-15 19:30:40,703 - INFO - Fold 4, Epoch 2/100 | Train Loss: 0.4823, Train Balanced Accuracy: 0.7593 | Val Loss: 2.6646, Val Balanced Accuracy: 0.6055
Fold 4 - Epoch 3/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.78it/s]
2025-02-15 19:30:44,906 - INFO - Fold 4, Epoch 3/100 | Train Loss: 0.4005, Train Balanced Accuracy: 0.8250 | Val Loss: 1.0717, Val Balanced Accuracy: 0.6643
Fold 4 - Epoch 4/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.07it/s]
2025-02-15 19:30:49,105 - INFO - Fold 4, Epoch 4/100 | Train Loss: 0.3153, Train Balanced Accuracy: 0.8697 | Val Loss: 1.2432, Val Balanced Accuracy: 0.7430
Fold 4 - Epoch 5/100 - Validation: 100%|████████

Loaded pretrained weights for efficientnet-b0


Fold 5 - Epoch 1/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  7.94it/s]
2025-02-15 19:37:28,716 - INFO - Fold 5, Epoch 1/100 | Train Loss: 0.6376, Train Balanced Accuracy: 0.6176 | Val Loss: 0.5471, Val Balanced Accuracy: 0.6476
Fold 5 - Epoch 2/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.54it/s]
2025-02-15 19:37:32,813 - INFO - Fold 5, Epoch 2/100 | Train Loss: 0.4505, Train Balanced Accuracy: 0.7892 | Val Loss: 0.7574, Val Balanced Accuracy: 0.7086
Fold 5 - Epoch 3/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.56it/s]
2025-02-15 19:37:36,914 - INFO - Fold 5, Epoch 3/100 | Train Loss: 0.3898, Train Balanced Accuracy: 0.8317 | Val Loss: 0.4141, Val Balanced Accuracy: 0.6989
Fold 5 - Epoch 4/100 - Validation: 100%|██████████| 5/5 [00:00<00:00,  8.02it/s]
2025-02-15 19:37:41,048 - INFO - Fold 5, Epoch 4/100 | Train Loss: 0.3026, Train Balanced Accuracy: 0.8807 | Val Loss: 0.4951, Val Balanced Accuracy: 0.7149
Fold 5 - Epoch 5/100 - Validation: 100%|████████

## Inference (Individually on each checkpoint)

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/Mitosis_Detection/MICCAI25/AMI-Br_dataset/Test_Held_out/Images', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate each fold model
num_folds = 5
fold_accuracies = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue

    logger.info(f"Evaluating Fold {fold + 1} Model")
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_preds = []
    test_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            test_preds.extend((outputs > 0.5).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(test_targets, test_preds)
    fold_accuracies.append(balanced_acc)
    logger.info(f"Fold {fold + 1} - Test Balanced Accuracy: {balanced_acc:.4f}")

# Calculate and log overall performance
if fold_accuracies:
    average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    logger.info(f"Average Test Balanced Accuracy across all folds: {average_accuracy:.4f}")


2025-02-15 19:49:52,803 - INFO - Evaluating Fold 1 Model


Loaded pretrained weights for efficientnet-b0


2025-02-15 19:49:53,791 - INFO - Fold 1 - Test Balanced Accuracy: 0.7474
2025-02-15 19:49:53,791 - INFO - Evaluating Fold 2 Model


Loaded pretrained weights for efficientnet-b0


2025-02-15 19:49:54,648 - INFO - Fold 2 - Test Balanced Accuracy: 0.7533
2025-02-15 19:49:54,648 - INFO - Evaluating Fold 3 Model


...

2025-02-15 19:49:55,542 - INFO - Fold 3 - Test Balanced Accuracy: 0.7586
2025-02-15 19:49:55,543 - INFO - Evaluating Fold 4 Model


Loaded pretrained weights for efficientnet-b0


2025-02-15 19:49:56,395 - INFO - Fold 4 - Test Balanced Accuracy: 0.7260
2025-02-15 19:49:56,396 - INFO - Evaluating Fold 5 Model


Loaded pretrained weights for efficientnet-b0


2025-02-15 19:49:57,150 - INFO - Fold 5 - Test Balanced Accuracy: 0.7895
2025-02-15 19:49:57,151 - INFO - Average Test Balanced Accuracy across all folds: 0.7549


In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/Mitosis_Detection/MICCAI25/AMI-Br_dataset/Test_Held_out/Images', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate each fold model
num_folds = 5
fold_accuracies = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue

    logger.info(f"Evaluating Fold {fold + 1} Model")
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_preds = []
    test_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            test_preds.extend((outputs > 0.5).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(test_targets, test_preds)
    fold_accuracies.append(balanced_acc)
    logger.info(f"Fold {fold + 1} - Test Balanced Accuracy: {balanced_acc:.4f}")

# Calculate and log overall performance
if fold_accuracies:
    average_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    logger.info(f"Average Test Balanced Accuracy across all folds: {average_accuracy:.4f}")
    logger.info(f"Standard Deviation of Balanced Accuracy: {std_accuracy:.4f}")


2025-02-26 14:50:53,678 - INFO - Evaluating Fold 1 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:51:03,413 - INFO - Fold 1 - Test Balanced Accuracy: 0.7474
2025-02-26 14:51:03,414 - INFO - Evaluating Fold 2 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:51:13,580 - INFO - Fold 2 - Test Balanced Accuracy: 0.7533
2025-02-26 14:51:13,581 - INFO - Evaluating Fold 3 Model


...

2025-02-26 14:51:22,790 - INFO - Fold 3 - Test Balanced Accuracy: 0.7586
2025-02-26 14:51:22,791 - INFO - Evaluating Fold 4 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:51:33,242 - INFO - Fold 4 - Test Balanced Accuracy: 0.7260
2025-02-26 14:51:33,245 - INFO - Evaluating Fold 5 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:51:43,019 - INFO - Fold 5 - Test Balanced Accuracy: 0.7895
2025-02-26 14:51:43,020 - INFO - Average Test Balanced Accuracy across all folds: 0.7549
2025-02-26 14:51:43,021 - INFO - Standard Deviation of Balanced Accuracy: 0.0205


## Majority Vote Ensembling

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/Mitosis_Detection/MICCAI25/AMI-Br_dataset/Test_Held_out/Images', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate ensembled model using Majority Voting
num_folds = 5
models = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue
    
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    models.append(model)

if not models:
    logger.error("No valid models loaded for ensembling.")
    exit()

logger.info("Performing majority voting ensembling inference")

test_preds = []
test_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        all_model_preds = []
        for model in models:
            outputs = model(images)
            preds = (outputs > 0.5).cpu().numpy()
            all_model_preds.append(preds)
        
        # Majority Voting
        all_model_preds = np.array(all_model_preds)  # Shape: (num_models, batch_size, 1)
        majority_preds = np.round(np.mean(all_model_preds, axis=0))  # Compute majority vote

        test_preds.extend(majority_preds)
        test_targets.extend(labels.cpu().numpy())

balanced_acc = balanced_accuracy_score(test_targets, test_preds)
logger.info(f"Majority Voting Ensembled Test Balanced Accuracy: {balanced_acc:.4f}")


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


2025-02-15 19:52:47,830 - INFO - Performing majority voting ensembling inference


Loaded pretrained weights for efficientnet-b0


2025-02-15 19:52:49,962 - INFO - Majority Voting Ensembled Test Balanced Accuracy: 0.7888


## AtNorM-MD

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/MICCAI25_Github/datasets/AtNorM-MD', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate ensembled model using Majority Voting
num_folds = 5
models = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue
    
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    models.append(model)

if not models:
    logger.error("No valid models loaded for ensembling.")
    exit()

logger.info("Performing majority voting ensembling inference")

test_preds = []
test_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        all_model_preds = []
        for model in models:
            outputs = model(images)
            preds = (outputs > 0.5).cpu().numpy()
            all_model_preds.append(preds)
        
        # Majority Voting
        all_model_preds = np.array(all_model_preds)  # Shape: (num_models, batch_size, 1)
        majority_preds = np.round(np.mean(all_model_preds, axis=0))  # Compute majority vote

        test_preds.extend(majority_preds)
        test_targets.extend(labels.cpu().numpy())

balanced_acc = balanced_accuracy_score(test_targets, test_preds)
logger.info(f"Majority Voting Ensembled Test Balanced Accuracy: {balanced_acc:.4f}")


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


2025-02-20 12:11:15,040 - INFO - Performing majority voting ensembling inference
2025-02-20 12:11:20,778 - INFO - Majority Voting Ensembled Test Balanced Accuracy: 0.7147


## AtNorM-Br

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/MICCAI25_Github/datasets/AtNorM-Br', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate ensembled model using Majority Voting
num_folds = 5
models = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue
    
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    models.append(model)

if not models:
    logger.error("No valid models loaded for ensembling.")
    exit()

logger.info("Performing majority voting ensembling inference")

test_preds = []
test_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        all_model_preds = []
        for model in models:
            outputs = model(images)
            preds = (outputs > 0.5).cpu().numpy()
            all_model_preds.append(preds)
        
        # Majority Voting
        all_model_preds = np.array(all_model_preds)  # Shape: (num_models, batch_size, 1)
        majority_preds = np.round(np.mean(all_model_preds, axis=0))  # Compute majority vote

        test_preds.extend(majority_preds)
        test_targets.extend(labels.cpu().numpy())

balanced_acc = balanced_accuracy_score(test_targets, test_preds)
logger.info(f"Majority Voting Ensembled Test Balanced Accuracy: {balanced_acc:.4f}")


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


2025-02-26 17:06:11,836 - INFO - Performing majority voting ensembling inference
2025-02-26 17:06:21,036 - INFO - Majority Voting Ensembled Test Balanced Accuracy: 0.6940


## AtNorM-MD

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/MICCAI25_Github/datasets/AtNorM-MD', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate each fold model
num_folds = 5
fold_accuracies = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue

    logger.info(f"Evaluating Fold {fold + 1} Model")
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_preds = []
    test_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            test_preds.extend((outputs > 0.5).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(test_targets, test_preds)
    fold_accuracies.append(balanced_acc)
    logger.info(f"Fold {fold + 1} - Test Balanced Accuracy: {balanced_acc:.4f}")

# Calculate and log overall performance
if fold_accuracies:
    average_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    logger.info(f"Average Test Balanced Accuracy across all folds: {average_accuracy:.4f}")
    logger.info(f"Standard Deviation of Balanced Accuracy: {std_accuracy:.4f}")


2025-02-26 14:55:47,676 - INFO - Evaluating Fold 1 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:56:00,756 - INFO - Fold 1 - Test Balanced Accuracy: 0.7132
2025-02-26 14:56:00,757 - INFO - Evaluating Fold 2 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:56:12,372 - INFO - Fold 2 - Test Balanced Accuracy: 0.6623
2025-02-26 14:56:12,373 - INFO - Evaluating Fold 3 Model


...

2025-02-26 14:56:25,406 - INFO - Fold 3 - Test Balanced Accuracy: 0.6718
2025-02-26 14:56:25,406 - INFO - Evaluating Fold 4 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:56:38,362 - INFO - Fold 4 - Test Balanced Accuracy: 0.6591
2025-02-26 14:56:38,363 - INFO - Evaluating Fold 5 Model


Loaded pretrained weights for efficientnet-b0


2025-02-26 14:56:50,779 - INFO - Fold 5 - Test Balanced Accuracy: 0.7024
2025-02-26 14:56:50,780 - INFO - Average Test Balanced Accuracy across all folds: 0.6818
2025-02-26 14:56:50,781 - INFO - Standard Deviation of Balanced Accuracy: 0.0219


## AtNorM-Br

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import balanced_accuracy_score
from efficientnet_pytorch import EfficientNet
import os
from PIL import Image
import logging
import numpy as np

# Setup logging
log_file = "efficientnet_test_inference.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Define transformations for test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load test data
def load_test_data(folder_path, classes):
    images = []
    labels = []
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

    for cls_name in classes:
        class_folder = os.path.join(folder_path, cls_name)
        if not os.path.isdir(class_folder):
            continue
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            if os.path.isfile(file_path):
                images.append(file_path)
                labels.append(class_to_idx[cls_name])

    return images, labels

# Custom Dataset class
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the model class
class BinaryEfficientNet(nn.Module):
    def __init__(self):
        super(BinaryEfficientNet, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Sequential(
            nn.Linear(self.model._fc.in_features, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Load test dataset
classes = ['Atypical', 'Normal']
test_images, test_labels = load_test_data('/data/MICCAI25_Github/datasets/AtNorM-Br', classes)
test_dataset = TestDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate each fold model
num_folds = 5
fold_accuracies = []

for fold in range(num_folds):
    model_path = f'Ami-br_efficientnet_master_baseline_fold{fold + 1}_best.pth'
    if not os.path.exists(model_path):
        logger.warning(f"Model checkpoint not found: {model_path}")
        continue

    logger.info(f"Evaluating Fold {fold + 1} Model")
    model = BinaryEfficientNet().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    test_preds = []
    test_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            test_preds.extend((outputs > 0.5).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    balanced_acc = balanced_accuracy_score(test_targets, test_preds)
    fold_accuracies.append(balanced_acc)
    logger.info(f"Fold {fold + 1} - Test Balanced Accuracy: {balanced_acc:.4f}")

# Calculate and log overall performance
if fold_accuracies:
    average_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    logger.info(f"Average Test Balanced Accuracy across all folds: {average_accuracy:.4f}")
    logger.info(f"Standard Deviation of Balanced Accuracy: {std_accuracy:.4f}")


2025-02-27 14:53:54,926 - INFO - Evaluating Fold 1 Model


Loaded pretrained weights for efficientnet-b0


2025-02-27 14:53:55,938 - INFO - Fold 1 - Test Balanced Accuracy: 0.6777
2025-02-27 14:53:55,939 - INFO - Evaluating Fold 2 Model


Loaded pretrained weights for efficientnet-b0


2025-02-27 14:53:56,619 - INFO - Fold 2 - Test Balanced Accuracy: 0.6822
2025-02-27 14:53:56,619 - INFO - Evaluating Fold 3 Model


...

2025-02-27 14:53:57,273 - INFO - Fold 3 - Test Balanced Accuracy: 0.6640
2025-02-27 14:53:57,274 - INFO - Evaluating Fold 4 Model


Loaded pretrained weights for efficientnet-b0


2025-02-27 14:53:57,950 - INFO - Fold 4 - Test Balanced Accuracy: 0.6910
2025-02-27 14:53:57,951 - INFO - Evaluating Fold 5 Model


Loaded pretrained weights for efficientnet-b0


2025-02-27 14:53:58,637 - INFO - Fold 5 - Test Balanced Accuracy: 0.7080
2025-02-27 14:53:58,638 - INFO - Average Test Balanced Accuracy across all folds: 0.6846
2025-02-27 14:53:58,639 - INFO - Standard Deviation of Balanced Accuracy: 0.0146
